In [2]:
# Skriptas rpeaks automatiniam ir rankiniam anotatacijų atstatymui.
# Skirtas Zive EKG įrašams su masiškoms rpeaks klaidomis.
# 
# Planas:
# - pasiruošimas
# - užduodamas failo pavadinimas 
# - nuskaitome užduotą įrašą ir jo atributus
# - atspausdiname EKG vietas su rankinėmis S ir V anotacijomis
# - sukuriame naują json failą, nukopijuojame iš seno failo, kas tinka
# - praleidžiame Neurokit, surandame rpeaks
# - suformuojame naują 'rpeaks' sekciją su 'N"
# - rankomis sudedame iš atspausdinto sąrašo S ir V anotacijas,
# - pataisome rankomis sekciją 'rpeakAnnotCounts'
# - įrašome sutvarkytą json į duomenų saugyklą
# - patikriname , kaip tas atrodo su 1_ziveo_visual_rpeaks_sulyginimas_v1.ipynb


import numpy as np
import neurokit2 as nk
from neurokit2 import signal_filter

# import math
import pandas as pd
import os, sys, shutil
import json
from pathlib import Path

from vertinimas_util import zive_read_file_1ch, AnalyseHeartrate, zive_read_df_data
from vertinimas_util import anotacijos, get_symbol_list, get_seq_start_end

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////
# 
#       1 Atrinktas failas

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI\\Data\\DI'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# arba variantas: Windows, be Docker 
# Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'

# Vietinės talpyklos aplankas 
db_folder = 'DUOM_2022_RUDUO_2'

#  Nuoroda į aplanką su ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_selected')

print("\nScriptas ZIVE EKG filtravimo vizualiniam įvertinimui")
print("Duomenų rinkinys: ", rec_dir)

#       2 Atrinktas failas

# Atrinkti failai pasižiūrėjimui
list = ['1637624.073'] # nesutampančių 1481,  taisomas specialiomis priemonėmis - pataisyta
list = ['1637621.569'] # nesutampančių 1354,  taisomas specialiomis priemonėmis - pataisyta
list = ['1637630.258'] # nesutampančių 1327,  taisomas specialiomis priemonėmis - pataisyta
list = ['1638807.019'] # nesutampančių 1192,  taisomas specialiomis priemonėmis - pataisyta

file_name = list[0]
print(f"\nZive įrašas:  {file_name:>2}")


#       3 nuskaitome užduotą įrašą ir jo atributus

# filepath = Path(rec_dir, file_name + '.json')
df_rpeaks = zive_read_df_data(rec_dir, file_name, 'rpeaks')
if (df_rpeaks.empty == True):
    print(f'Annotation file for {file_name} does not exist or nrpeaks is empty!')
# else:
    # atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    # atr_symbol = df_rpeaks['annotationValue'].to_numpy()


#       3 nuskaitome užduotą įrašą ir jo atributus

# print(f"rpeaks iš json: {len(atr_sample)}")

# print(atr_sample[:10])
# print(atr_symbol[:10])

counts = df_rpeaks['annotationValue'].value_counts(sort = True, ascending = True)
# print(counts)
print("\nINFORMACIJA IŠ TAISOMO JSON FAILO:")
print(f"\nrpeaks iš json failos skaičius: {len(df_rpeaks)}")

#       4 atspausdiname EKG vietas su rankinėmis S ir V anotacijomis
lst = {}
for idx,name in enumerate(df_rpeaks['annotationValue'].value_counts().index.tolist()):
    counts = df_rpeaks['annotationValue'].value_counts()[idx]
    lst[name] = counts
print("\nAnotacijų įraše (F ir U ignoruojamos):")
print(lst)

# Galime pažiūrėti, kur yra anotacijos
anot_S = anotacijos(df_rpeaks,"S") 
print("\nAnotacijų S vietos:", anot_S)
anot_V = anotacijos(df_rpeaks,"V")
print("\nAnotacijos V:", anot_V)

#       5 sukuriame naują json ir nukopijuojame tinkamus objektus
# Loading a JSON File in Python – How to Read and Parse JSON
# https://www.freecodecamp.org/news/loading-a-json-file-in-python-how-to-read-and-parse-json/

with open(filepath) as user_file:
  json_orig = json.load(user_file)
print()
# print(json_orig)

json_new = {}
json_new["recordingId"] = json_orig["recordingId"]
json_new["channelCount"] = json_orig["channelCount"]
json_new["userId"] = json_orig["userId"]
json_new["flags"] = json_orig["flags"]
json_new["noises"] = json_orig["noises"]


#       6 Nuskaitome įrašą ir surandame rpeaks su Neurokit
        # pakartojant Zive skriptą iš analysis.py ir heartrate_analysis.py
filepath = Path(rec_dir, file_name)
signal_raw = zive_read_file_1ch(filepath)
ecg_signal_df = pd.DataFrame(signal_raw, columns=['orig'])
analysis_results = AnalyseHeartrate(ecg_signal_df)
rpeaks_from_signal = analysis_results['rpeaks']

print("\nINFORMACIJA NAUJAM JSON FAILUI:")
print(f"\nNaujų rpeaks iš signalo skaičius: {len(rpeaks_from_signal)}")

#       7 Suformuojame naują objektą "rpeaks"
rpeaks_new = []
for idx in range(len(rpeaks_from_signal)):
    rpeaks_new.append({"sampleIndex": int(rpeaks_from_signal[idx]),
            "annotationValue": "N"})

#       8 Į atnaujintą rpeaks_new įdedame išsaugotas anotacijas S ir V
def search(sampleIndex, test_list):
    return [element for element in test_list if element['sampleIndex'] == sampleIndex]
 
# https://www.geeksforgeeks.org/python-find-dictionary-matching-value-in-list/
# https://realpython.com/iterate-through-dictionary-python/

print("\nAtstatomos anotacijos:\n")
sum_anot_V = 0
if (anot_V):
    for sampleIndex, annotationValue in anot_V.items():
        if (sampleIndex in rpeaks_from_signal):
            for element in rpeaks_new:
	            if (element['sampleIndex']== sampleIndex):
                        sum_anot_V += 1
                        element['annotationValue'] = 'V'
                        print(element)
    print("\nAtstatyta rankinių anotacijų V : ", sum_anot_V)
if (sum_anot_V != len(anot_V)):
    print("\nOriginalių ir atstatytų anotacijų V skaičius nesutampa!")

sum_anot_S = 0
if (anot_S):
    for sampleIndex, annotationValue in anot_S.items():
        if (sampleIndex in rpeaks_from_signal):
            for element in rpeaks_new:
	            if (element['sampleIndex']== sampleIndex):
                        sum_anot_S += 1
                        element['annotationValue'] = 'S'
                        print(element)
    print("\nAtstatyta rankinių anotacijų S : ", sum_anot_V)
if (sum_anot_S != len(anot_S)):
    print("\nOriginalių ir atstatytų anotacijų S skaičius nesutampa!")



# Atnaujiname "rpeakAnnotationCounts"
print("\nOriginalus json objektas rpeakAnnotationCounts:")
rpeakAnnotationCounts = json_orig["rpeakAnnotationCounts"]
print(rpeakAnnotationCounts)
rpeakAnnotationCounts['N'] = len(rpeaks_from_signal)
if (anot_S):
    rpeakAnnotationCounts['S'] = len(anot_S)
if (anot_V):
    rpeakAnnotationCounts['V'] = len(anot_V)
print("\npakeičiamas į:")
json_new["rpeakAnnotationCounts"] = rpeakAnnotationCounts
print(rpeakAnnotationCounts)

# print("\n")
# print(rpeaks_new)

#       9 Atnaujintą rpeaks įdedame į atributų failą
json_new["rpeaks"] = rpeaks_new

#       10 Įrašome atributų failą į saugyklą
filepath = Path(rec_dir, file_name + '_new' + '.json')
with open(filepath, "w") as outfile:
    json.dump(json_new, outfile)
print("\nAtnaujintas json failas įrašytas į:", filepath)

#       11 Įrašome į saugyklą EKG failo kopiją su kitu atnaujintu pavadinimu
src = Path(rec_dir, file_name)
dst = Path(rec_dir, file_name + '_new')
shutil.copy2(src, dst)
print("Tiksli EKG įrašo kopija įrašyta į", dst)
print("\n")
# print(json_new)

OS in my system :  linux

Scriptas ZIVE EKG filtravimo vizualiniam įvertinimui
Duomenų rinkinys:  /home/kesju/DI/DUOM_2022_RUDUO_2/records_selected

Zive įrašas:  1638807.019

INFORMACIJA IŠ TAISOMO JSON FAILO:

rpeaks iš json failos skaičius: 628

Anotacijų įraše (F ir U ignoruojamos):
{'N': 627, 'V': 1}

Anotacijų S vietos: {}

Anotacijos V: {85211: 'V'}


INFORMACIJA NAUJAM JSON FAILUI:

Naujų rpeaks iš signalo skaičius: 570

Atstatomos anotacijos:

{'sampleIndex': 85211, 'annotationValue': 'V'}

Atstatyta rankinių anotacijų V :  1

Originalus json objektas rpeakAnnotationCounts:
{'__info': 'These include BOTH annotations created by analysis software (default R-peaks annotated with N) AND manually created and/or altered annotations', 'N': 627, 'V': 1}

pakeičiamas į:
{'__info': 'These include BOTH annotations created by analysis software (default R-peaks annotated with N) AND manually created and/or altered annotations', 'N': 570, 'V': 1}

Atnaujintas json failas įrašytas į: /home/k